#### Dataset Correction
- To be used when correcting format of datasets from Cycle 2

In [14]:
import pandas as pd
sep = "\t" #,

# Correction Procedure (clear input column)
def correction_dataset(df, correction_procedure : str = "A", url : str = ""):
    if (correction_procedure == "A"):
        # Correction Procedure A : 1) Convert Instruction Column Contents to Instruction+Input, Clear Input Contents
        i = 0
        while i<len(df):
            df['instruction'][i] = df['instruction'][i].replace(df['instruction'][i], df['instruction'][i] + " " + df['input'][i])
            df['input'][i] = df['input'][i].replace(df['input'][i], "")
            i+=1
    if (correction_procedure == "B"):
        i = 0
        while i<len(df):
            df['instruction'][i] = df['instruction'][i].replace(df['instruction'][i], df['instruction'][i] + " " + url)
            i+=1
    return df

csvs = ["dostgovph.csv"]
urls = ["https://www.dost.gov.ph/"]
batch = True
setting = "B"

if (batch == True):
    i = 0
    while i<len(csvs):
        df = pd.read_csv(csvs[i], sep=sep)
        print(df)
        df = correction_dataset(df, correction_procedure=setting, url=urls[i])
        print(df)
        df.to_csv(f"{csvs[i].replace(".csv", "")}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)
        i += 1
else:
    df = pd.read_csv(csvs[0], sep=sep)
    print(df)
    df = correction_dataset(df, correction_procedure=setting, url=urls[0])
    print(df)
    df.to_csv(f"{csvs[0].replace(".csv", "")}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)


                                                output  input  \
0    Verify that the Accessibility Button on the Do...    NaN   
1    Verify that the Accessibility Button on the Do...    NaN   
2    Verify that the Accessibility Button on the Do...    NaN   
3    Verify that the 'Pause' button on the Dost web...    NaN   
4    Verify that the "Pause" button on the Dost web...    NaN   
..                                                 ...    ...   
264  Verify that the search input field on the dost...    NaN   
265  Verify that the search input field on the Dost...    NaN   
266  Verify that the search input field on the dost...    NaN   
267  Verify that the search input field on the dost...    NaN   
268  Verify that the search input field on the dost...    NaN   

                                           instruction  
0    Generate test case for the following UI elemen...  
1    Generate test case for the following UI elemen...  
2    Generate test case for the following UI ele

C:\Users\user\AppData\Local\Temp\ipykernel_18280\3903927304.py:16: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['instruction'][i] = df['instruction'][i].replace(df['instruction'][i], df['instruction'][i] + " " + url)
C:\Users\user\AppDat

                                                output  input  \
0    Verify that the Accessibility Button on the Do...    NaN   
1    Verify that the Accessibility Button on the Do...    NaN   
2    Verify that the Accessibility Button on the Do...    NaN   
3    Verify that the 'Pause' button on the Dost web...    NaN   
4    Verify that the "Pause" button on the Dost web...    NaN   
..                                                 ...    ...   
264  Verify that the search input field on the dost...    NaN   
265  Verify that the search input field on the Dost...    NaN   
266  Verify that the search input field on the dost...    NaN   
267  Verify that the search input field on the dost...    NaN   
268  Verify that the search input field on the dost...    NaN   

                                           instruction  
0    Generate test case for the following UI elemen...  
1    Generate test case for the following UI elemen...  
2    Generate test case for the following UI ele